# PRACTICA 1 - GRIPPER FLOTANTE 
## Iniciando la simulacion de Gazebo

En primer lugar, es necesario iniciar la simulación de Gazebo para el entorno del gripper flotante. 
> **Gripper asignado:** `robotiq`  
> **Objeto asignado:** `driller_small`


Para facilitar el proceso, se ha creado un archivo de lanzamiento (launch file) que configura automáticamente el entorno de simulación con el gripper y el objeto asignados.

1. Abre una Nueva Terminal: Primero, necesitas abrir una nueva ventana de terminal en tu sistema (Ctrl + Alt + T).

2. Ejecuta el Comando de Lanzamiento: En la terminal recién abierta, ejecuta el siguiente comando:



``` shellscript
user@pc:~$ roslaunch manipulacion_pkg main_simulation.launch
```

In [2]:
# Definir variables globales con el nombre del gripper y objeto
NOMBRE_GRIPPER_GAZEBO = "gripper"     
NOMBRE_OBSTACULO_GAZEBO = "red_obstacle"
NOMBRE_OBJETO = "driller_small"
NOMBRE_GRIPPER = "robotiq"

In [3]:
import manipulacion_lib 
import rospy 
import PyKDL 

# Inicializar el nodo de ROS si aún no se ha inicializado
if not rospy.get_node_uri():
    rospy.init_node('gripper_flotante_gazebo', anonymous=True, 
                    log_level=rospy.WARN)

# Creación de una instancia para controlar un gripper flotante en Gazebo
simulacion_gripper_flotante = manipulacion_lib.SimulacionGripperFlotante(
                                nombre_gripper_gazebo=NOMBRE_GRIPPER_GAZEBO)

# Esta instancia permite interactuar con un gripper en la simulación de Gazebo,
# especificando su nombre
# Independientemente del gripper asignado para la práctica, se utiliza el 
# nombre "gripper" para referenciarlo en la simulación de Gazebo.


# Obtener la pose (posición y orientación) de un objeto específico en el Gazebo
# Se obtiene la pose del objeto con respecto al sistema de referencia global (world)
pose_objeto_world = simulacion_gripper_flotante.obtener_pose_objeto(
                    nombre_objeto_gazebo=NOMBRE_OBJETO)

print("Pose objeto con respecto a world")
print(pose_objeto_world)

# Fijamos una pose relativa del gripper con respecto al objeto
# Pose del gripper con respecto al objeto, especificando una
# traslación de 0.2m en X y 0.1m en Z
pose_gripper_objeto = PyKDL.Frame(PyKDL.Rotation.Quaternion(1.0, 0, 0.0, 0.0), 
                                  PyKDL.Vector(0.2, 0, 0.1))

print("Pose gripper con respecto a objeto")
print(pose_gripper_objeto)

# Calcular la pose del gripper en el sistema de referencia global a partir 
# de su pose relativa al objeto
# Se realiza una transformación de coordenadas para obtener la pose del 
# gripper en el sistema global
pose_gripper_world = pose_objeto_world * pose_gripper_objeto
print("Pose gripper con respecto a world: ")
print( pose_gripper_world)
# Fijar la pose del gripper en la simulación de Gazebo
simulacion_gripper_flotante.fijar_pose_gripper(
                    pose_gripper_world=pose_gripper_world)
# Actualiza la posición y orientación del gripper en Gazebo según
# la pose calculada

Pose objeto con respecto a world
[[    0.695372,   -0.717993,   0.0307185;
     0.717384,    0.696049,   0.0296018;
   -0.0426355,  0.00145266,     0.99909]
[    0.294201,    0.210765,   0.0829699]]
Pose gripper con respecto a objeto
[[           1,           0,           0;
            0,          -1,           0;
            0,           0,          -1]
[         0.2,           0,         0.1]]
Pose gripper con respecto a world: 
[[    0.695372,    0.717993,  -0.0307185;
     0.717384,   -0.696049,  -0.0296018;
   -0.0426355, -0.00145266,    -0.99909]
[    0.436347,    0.357202,    0.174352]]


True

In [4]:
offset = 0.2 # Definimos un offset de 0.2 metros

# Creamos una nueva pose de tipo Frame de PyKDL que no altera la rotación 
# (identidad) y desplaza el gripper 0.2 metros a lo largo del eje X local.
pose_nueva_gripper = PyKDL.Frame(PyKDL.Rotation(), PyKDL.Vector(offset, 0, 0 ))

# Multiplicamos la pose actual del gripper en el mundo por la transformación
# que acabamos de crear para obtener la nueva pose en el sistema de coordenadas 
# del mundo.
pose_nueva_world = pose_gripper_world*pose_nueva_gripper

# Fijamos la nueva pose del gripper en la simulación de Gazebo.
simulacion_gripper_flotante.fijar_pose_gripper(
                            pose_gripper_world=pose_nueva_world)

True

# Detectar colisiones entre el gripper y el entorno

In [5]:
from manipulacion_lib import Obstaculo, DetectorColisionesGripperFlotante
obstaculo_rojo = Obstaculo('cubo', [0.75,0,0,1,0,0,0], [0.3,1,0.6], 'obstaculo')
suelo = Obstaculo('cubo', [0,0,0,1,0,0,0], [0.01,0.003,-0.01], 'suelo')

obj_x, obj_y, obj_z = pose_objeto_world.p
obj_qx, obj_qy, obj_qz, obj_qw = pose_objeto_world.M.GetQuaternion() 
objeto = Obstaculo('cubo', [obj_x, obj_y, obj_z, obj_qx, obj_qy, obj_qz, obj_qw], [0.141,0.095,0.226], 'objeto')
obstaculos = [obstaculo_rojo, suelo, objeto]

detector_colisiones = DetectorColisionesGripperFlotante(NOMBRE_GRIPPER, obstaculos)

# Obtener la matriz de rotación del frame
rotation = pose_gripper_world.M

# Convertir la matriz de rotación a cuaternión
qx, qy, qz, qw = rotation.GetQuaternion()
pose_gripper = [pose_gripper_world.p.x(), pose_gripper_world.p.y(),
                pose_gripper_world.p.z(),
                qx, qy, qz, qw]
print("Colisiona en la pose actual:", detector_colisiones.hay_colision(pose_gripper))

/workspaces/Handling_RoboticMaster/manipulacion_ws/src/manipulacion_pkg/urdf/robotiq.urdf
Colisiona en la pose actual: True


/workspaces/Handling_RoboticMaster/manipulacion_ws/src/manipulacion_pkg/scripts/manipulacion_lib.py:1015: UserWarning: This function has been marked as deprecated and will be removed in a future release.
  return pin.GeometryObject(


# Mover Articulaciones del Gripper

In [6]:
# Configurar el gripper basado en el archivo YAML específico para su tipo.
# Esto incluye cargar las configuraciones iniciales como nombres de articulaciones y posiciones predeterminadas.
simulacion_gripper_flotante.configurar_gripper()

# Abrir completamente el gripper antes de intentar cualquier operación de agarre.
# Es una práctica común asegurarse de que el gripper no esté restringido o en una posición que podría interferir con el objeto a agarrar.
simulacion_gripper_flotante.abrir_gripper()

# Pequeña pausa para asegurar que la acción de abrir se ha completado.
rospy.sleep(1)

# Obtener el tipo de gripper actual para aplicar la configuración de posiciones de articulaciones correcta.
tipo_gripper = simulacion_gripper_flotante.get_tipo_gripper()

from practica1.auxiliar import set_gripper_pos
set_gripper_pos(simulacion_gripper_flotante, mode="close")

Gripper posicion articulaciones:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Cerrando gripper...
Gripper posicion articulaciones:  [0.4, 0.4, 0.4, 0, 0.4, 0.4, 0.4, 0, 0.4, 0.4, 0.4]
